section - importing necessary libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta
import pytz

In [2]:
API_KEY = '75926732325fd00c6922fa4365449e85'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

1.Fetch current data.

In [3]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city':data['name'],
      'current_temp':round(data['main']['temp']),
      'feels_like':round(data['main']['feels_like']),
      'temp_min':round(data['main']['temp_min']),
      'temp_max':round(data['main']['temp_max']),
      'humidity':round(data['main']['humidity']),
      'description':data['weather'][0]['description'],
      'country':data['sys']['country'],
      'wind_gust_dir':data['wind']['deg'],
      'pressure':data['main']['pressure'],
      'wind_gust_speed':data['wind']['speed']
  }

2. Read historical data.

In [4]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

3. Prepare data for training.

In [5]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  x = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
  y = data['RainTomorrow']

  return x,y,le



4. Train rain prediction model.

In [6]:
def train_rain_model(x,y):
  x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)
  model = RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  print("mean squared error for rain model")
  print(mean_squared_error(y_test,y_pred))
  return model

5. Prepare regression data.

In [7]:
def prepare_regression_data(data,feature):
  x,y = [],[]
  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])
  x = np.array(x).reshape(-1,1)
  y = np.array(y)
  return x,y

6. Train regression model.

In [8]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100,random_state=42)
  model.fit(x,y)
  return model

predict future.

In [9]:
def predict_future(model,current_value):
  predictions = [current_value]
  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:]

weather analysis function.

In [10]:
 def weather_view():
  city = input("enter city name:")
  current_weather = get_current_weather(city)
  historical_data = read_historical_data('weather.csv')
  x,y,le = prepare_data(historical_data)
  rain_model = train_rain_model(x,y)
  #map wind direction to campass points

  wind_deg = current_weather['wind_gust_dir'] % 360

  compass_points = [
        ("Ν", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("N№", 303.75, 326.25),
        ("N", 326.25, 348.75)
  ]

  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  curr_data = {
      'MinTemp':current_weather['temp_min'],
      'MaxTemp':current_weather['temp_max'],
      'WindGustDir':compass_direction_encoded,
      'WindGustSpeed':current_weather['wind_gust_speed'],
      'Humidity':current_weather['humidity'],
      'Pressure':current_weather['pressure'],
      'Temp':current_weather['current_temp'],
  }

  current_df = pd.DataFrame([curr_data])

  rain_prediction = rain_model.predict(current_df)[0]

  x_temp,y_temp = prepare_regression_data(historical_data,'Temp')

  x_hum,y_hum = prepare_regression_data(historical_data,'Humidity')

  temp_model = train_regression_model(x_temp,y_temp)

  hum_model = train_regression_model(x_hum,y_hum)

  future_temp = predict_future(temp_model,current_weather['temp_min'])

  future_humidity = predict_future(temp_model,current_weather['humidity'])

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now +timedelta(hours = 1)
  next_hour = next_hour.replace(minute = 0,second = 0,microsecond = 0)
  future_times = [(next_hour + timedelta(hours = i)).strftime("%H:00")for i in range(5)]
  print(f"city:{city},{current_weather['country']}")
  print(f"current temperature:{current_weather['current_temp']}°ca")
  print(f"feels like:{current_weather['feels_like']}°c")
  print(f"minimum temperature:{current_weather['temp_min']}°c")
  print(f"maximum temperature:{current_weather['temp_max']}°c")
  print(f"humidity:{current_weather['humidity']}%")
  print(f"weather prediction:{current_weather['description']}")
  print(f"rain prediction:{'yes' if rain_prediction else 'no'}")

  print("\nfuture temperature predictions:")
  for time,temp in zip(future_times,future_temp):
    print(f"{time}:{round(temp,1)}°c")

  print("\nfuture humidity predictions:")
  for time,humidity in zip(future_times,future_humidity):
    print(f"{time}:{round(humidity,1)}%")
weather_view()





mean squared error for rain model
0.1506849315068493
city:england,US
current temperature:14°ca
feels like:13°c
minimum temperature:14°c
maximum temperature:14°c
humidity:67%
weather prediction:overcast clouds
rain prediction:no

future temperature predictions:
12:00:13.9°c
13:00:15.8°c
14:00:14.7°c
15:00:12.4°c
16:00:14.4°c

future humidity predictions:
12:00:33.1%
13:00:25.1%
14:00:20.1%
15:00:16.6%
16:00:14.4%
